In [ ]:
using Gridap
using GridapMakie, CairoMakie, FileIO
using Gridap.FESpaces
using Gridap.ReferenceFEs
using Gridap.Arrays
using Gridap.Algebra
using Gridap.Geometry
using Gridap.Fields
using Gridap.CellData
using FillArrays
using Test
using InteractiveUtils
using PyPlot
using JSON3
using Dates
# using Interpolations

In [ ]:
include("Julia_functions/indicator_chi.jl")
include("Julia_functions/cost_fun.jl")
include("Julia_functions/gradient_descent.jl")
include("Julia_functions/solvers.jl")
include("Julia_functions/find.jl")
include("Julia_functions/res_plot.jl")
include("Julia_functions/get_domain.jl")
include("Julia_functions/get_price_temp.jl")
include("Julia_functions/post_processing.jl")

In [ ]:
# --- Problemparametere ---
h = 1.0
k = 1.0
ρ = 1.0
c = 1.0
θ = 0.5
Δt = 0.01
t0 = 0.0
tF = 1
h_wall = 1.0

# --- Manufactured solution og avledede størrelser ---
T_exact(x, t) = sin(x[1] + x[2] + t)
T_t(x, t) = cos(x[1] + x[2] + t)
ΔT(x, t) = -2 * sin(x[1] + x[2] + t)
Q(x, t) = T_t(x, t) - ΔT(x, t)


T_out_up(x, t) = cos(x[1] + x[2] + t) + T_exact(x, t)
T_out_up(t) = x -> T_out_up(x,t)

T_out_down(x, t) = -cos(x[1] + x[2] + t) + T_exact(x, t)
T_out_down(t) = x -> T_out_down(x,t)

T_out_left(x, t) = -cos(x[1] + x[2] + t) + T_exact(x, t)
T_out_left(t) = x -> T_out_left(x,t)

T_out_right(x, t) = cos(x[1] + x[2] + t) + T_exact(x, t)
T_out_right(t) = x -> T_out_right(x,t)


Qt(t) = x -> Q(x, t)

# --- Initialtilstand ---
TIni(x) = T_exact(x, t0)

# --- Geometri og mesh ---
domain = (0.0, 1.0, 0.0, 1.0)
partition = (30, 30)
model = CartesianDiscreteModel(domain, partition)

labels = get_face_labeling(model)
add_tag_from_tags!(labels, "left", [1,3,7])
add_tag_from_tags!(labels, "right", [2,4,8])
add_tag_from_tags!(labels, "bottom", [5])
add_tag_from_tags!(labels, "top", [6])

Ω = Triangulation(model)
Γ_up = BoundaryTriangulation(model, tags="top")
Γ_down = BoundaryTriangulation(model, tags="bottom")
Γ_left = BoundaryTriangulation(model, tags="left")
Γ_right = BoundaryTriangulation(model, tags="right")

order = 1
degree = 2 * order
dΩ = Measure(Ω, degree)
dΓ_up = Measure(Γ_up, degree)
dΓ_down = Measure(Γ_down, degree)
dΓ_left = Measure(Γ_left, degree)
dΓ_right = Measure(Γ_right, degree)

# --- Rom og FE-definisjon ---
reffe = ReferenceFE(lagrangian, Float64, order)

Testspace = TestFESpace(model, reffe; conformity=:H1)
Trialspace = TransientTrialFESpace(Testspace)

Uspace = FESpace(model, reffe, conformity=:H1)


# --- Svak form ---
a_dt(t, dT, v) = ∫(c * dT * v * ρ) * dΩ

a_SE(t, T, v) = ∫(k * ∇(T) ⋅ ∇(v)) * dΩ +
                ∫(h_wall * T * v)dΓ_up +
                ∫(h_wall * T * v)dΓ_down +
                ∫(h_wall * T * v)dΓ_left +
                ∫(h_wall * T * v)dΓ_right

l_SE(t, v) = ∫(Qt(t) * v) * dΩ +
             ∫(T_out_up(t) * v*h_wall)dΓ_up +
             ∫(T_out_down(t) *v*h_wall)dΓ_down +
             ∫(T_out_left(t) *v*h_wall)dΓ_left +
             ∫(T_out_right(t) * v*h_wall)dΓ_right

# --- Operator og løsning ---
ls = LUSolver()
solver = ThetaMethod(ls, Δt, θ)

op = TransientLinearFEOperator((a_SE, a_dt), l_SE, Trialspace, Testspace, constant_forms=(false, false))

T0 = interpolate(TIni, Trialspace)
T = solve(solver, op, t0, tF, T0)

# test = interpolate_everywhere(Tini, Uspace(t0))

uh = T
test(t) = x -> T_exact(x,t)

# if !isdir("tmp")
#     mkdir("tmp")
#   end
  
#   createpvd("results") do pvd
#     # pvd[0] = createvtk(Ω, "tmp/results_0" * ".vtu", cellfields=["u" => uh0])
#     for (tn, uhn) in uh

#         # TIni=interpolate_everywhere(T_exact, Uspace(tn))

#         pvd[tn] = createvtk(Ω, "tmp/results_$tn" * ".vtu", cellfields=["u" => uhn-test(tn)])
#         # pvd[tn] = createvtk(Ω, "tmp/results_$tn" * ".vtu", cellfields=["u" => test(tn)])

#     end
#   end

In [ ]:
T_exact_end(x) = sin(x[1] + x[2] + tF)
# T_exact_end1 = interpolate_everywhere(T_exact_end, Uspace(tF))

error = 0
for (t,TT) in T
    tmp = TT - T_exact_end
    error = ∑(∫(tmp*tmp)*dΩ)
end
println(error)

# tmp = T_exact_end - last(T)
# temp_difference = ∑(∫(tmp*tmp)*dΩ)

In [ ]:
# # Definer tidsstegene og løsningen
# T = [(t0, T0)]  # Start med initialbetingelsen
# for (t, TT) in zip(timesteps, computed_solutions)
#     push!(T, (t, FEFunction(Trialspace, copy(get_free_dof_values(TT)))))
# end

# # Lagre løsningene som VTK-filer og lag PVD-filen
# if !isdir("tmp_M")
#     mkdir("tmp_M")
# end

# createpvd("results_M") do pvd
#     # Lagre den initiale løsningen ved t = t0
#     pvd[0] = createvtk(Ω, "tmp_M/results_0.vtu", cellfields=["T" => T0])
    
#     # Lagre løsningene for hvert tidssteg
#     for (tn, (t, TT)) in enumerate(T)
#         # Hent FE-funksjonen for løsningen ved tidssteg t
#         Tn = FEFunction(Trialspace, copy(get_free_dof_values(TT)))
        
#         # Lagre løsningen som en VTK-fil
#         pvd[tn] = createvtk(Ω, "tmp_M/results_$tn.vtu", cellfields=["T" => Tn])
#     end
# end

